In [1]:
!nvidia-smi

Sun Jun  2 05:46:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install "transformers[sentencepiece]" datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.30.1
Uninstalling accelerate-0.30.1:
  Successfully uninstalled accelerate-0.30.1
  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)


In [4]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer, pipeline
from datasets import load_dataset, concatenate_datasets, load_metric
import pandas as pd
from tqdm import tqdm
import nltk


In [5]:
# Download NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#For Apple Silicon Users
# Set device for MPS (Apple Silicon) or CPU

#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#For Windows Users, isnce I am using Colab, I will be utilising this
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
# Function to load PEGASUS model and tokenizer
def load_pegasus_model(device):
    model_ckpt = "google/pegasus-cnn_dailymail"
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt, model_max_length = 1024)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
    return model, tokenizer

In [9]:
# Function to load BART model and tokenizer
def load_bart_model(device):
    model_ckpt = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_ckpt, model_max_length = 1024)
    model = BartForConditionalGeneration.from_pretrained(model_ckpt).to(device)
    return model, tokenizer

In [10]:
# Define dataset names, configurations, and trust_remote_code flags
datasets_info = [
    {"name": "cnn_dailymail", "config": "3.0.0", "trust_remote_code": False},
    {"name": "xsum", "config": None, "trust_remote_code": True},
    {"name": "multi_news", "config": None, "trust_remote_code": True},
    {"name": "gigaword", "config": None, "trust_remote_code": True},
    {"name": "big_patent", "config": "a", "trust_remote_code": True},
    {"name": "samsum", "config": None, "trust_remote_code": True},
]

In [11]:
# Load datasets
def load_datasets(dataset_info):
    datasets = []
    for dataset in dataset_info:
        name = dataset['name']
        config = dataset.get('config')
        trust_remote_code = dataset.get('trust_remote_code', False)
        ds = load_dataset(name, config, trust_remote_code=trust_remote_code) if config else load_dataset(name, trust_remote_code=trust_remote_code)
        datasets.append(ds)
    return datasets

In [12]:
# Standardize column names
def standardize_column_names(datasets, column_mappings):
    for i, dataset in enumerate(datasets):
        for split in dataset.keys():
            datasets[i][split] = dataset[split].rename_columns(column_mappings[i])
    return datasets

In [13]:
# Concatenate splits
def concatenate_splits(datasets):
    train_datasets = [dataset['train'].select(range(1000)) for dataset in datasets if 'train' in dataset]
    val_datasets = [dataset['validation'].select(range(100)) for dataset in datasets if 'validation' in dataset]
    test_datasets = [dataset['test'].select(range(100)) for dataset in datasets if 'test' in dataset]
    return concatenate_datasets(train_datasets), concatenate_datasets(val_datasets), concatenate_datasets(test_datasets)

In [14]:
# Define function to convert examples to features
def convert_examples_to_features(example_batch, tokenizer):
    input_encodings = tokenizer(example_batch['text'], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [15]:
# Preprocess data
def preprocess_data(dataset, tokenizer):
    return dataset.map(lambda x: convert_examples_to_features(x, tokenizer), batched=True)

In [16]:
# Load and prepare datasets
datasets = load_datasets(datasets_info)
column_mappings = [
    {'article': 'text', 'highlights': 'summary'},
    {'document': 'text', 'summary': 'summary'},
    {'document': 'text', 'summary': 'summary'},
    {'document': 'text', 'summary': 'summary'},
    {'description': 'text', 'abstract': 'summary'},
    {'dialogue': 'text', 'summary': 'summary'}
]
datasets = standardize_column_names(datasets, column_mappings)
train_dataset, val_dataset, test_dataset = concatenate_splits(datasets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
# Function to train and evaluate both models
def train_and_evaluate_models():
    # Train and evaluate PEGASUS
    pegasus_model, pegasus_tokenizer = load_pegasus_model(device)
    train_dataset_pegasus = preprocess_data(train_dataset, pegasus_tokenizer)
    val_dataset_pegasus = preprocess_data(val_dataset, pegasus_tokenizer)
    test_dataset_pegasus = preprocess_data(test_dataset, pegasus_tokenizer)

    seq2seq_data_collator = DataCollatorForSeq2Seq(pegasus_tokenizer, model=pegasus_model)
    trainer_args = TrainingArguments(
        output_dir='pegasus-summarizer', num_train_epochs=1, warmup_steps=100,
        per_device_train_batch_size=1, per_device_eval_batch_size=1,
        weight_decay=0.01, logging_steps=10,
        evaluation_strategy='steps', eval_steps=100, save_steps=500,
        gradient_accumulation_steps=8
    )
    trainer_pegasus = Trainer(model=pegasus_model, args=trainer_args,
                              tokenizer=pegasus_tokenizer, data_collator=seq2seq_data_collator,
                              train_dataset=train_dataset_pegasus,
                              eval_dataset=val_dataset_pegasus)
    trainer_pegasus.train()

    rouge_metric = load_metric('rouge')
    score_pegasus = calculate_metric_on_test_ds(test_dataset_pegasus, rouge_metric, pegasus_model, pegasus_tokenizer, batch_size=2)
    print(pd.DataFrame(dict((rn, score_pegasus[rn].mid.fmeasure) for rn in ["rouge1", "rouge2", "rougeL", "rougeLsum"]), index=['PEGASUS']))

    pegasus_model.save_pretrained("pegasus-summarizer-model")
    pegasus_tokenizer.save_pretrained("pegasus-tokenizer")

    # Train and evaluate BART
    bart_model, bart_tokenizer = load_bart_model(device)
    train_dataset_bart = preprocess_data(train_dataset, bart_tokenizer)
    val_dataset_bart = preprocess_data(val_dataset, bart_tokenizer)
    test_dataset_bart = preprocess_data(test_dataset, bart_tokenizer)

    seq2seq_data_collator = DataCollatorForSeq2Seq(bart_tokenizer, model=bart_model)
    trainer_args = TrainingArguments(
        output_dir='bart-summarizer', num_train_epochs=1, warmup_steps=100,
        per_device_train_batch_size=1, per_device_eval_batch_size=1,
        weight_decay=0.01, logging_steps=10,
        evaluation_strategy='steps', eval_steps=100, save_steps=500,
        gradient_accumulation_steps=8
    )
    trainer_bart = Trainer(model=bart_model, args=trainer_args,
                           tokenizer=bart_tokenizer, data_collator=seq2seq_data_collator,
                           train_dataset=train_dataset_bart,
                           eval_dataset=val_dataset_bart)
    trainer_bart.train()

    score_bart = calculate_metric_on_test_ds(test_dataset_bart, rouge_metric, bart_model, bart_tokenizer, batch_size=2)
    print(pd.DataFrame(dict((rn, score_bart[rn].mid.fmeasure) for rn in ["rouge1", "rouge2", "rougeL", "rougeLsum"]), index=['BART']))

    bart_model.save_pretrained("bart-summarizer-model")
    bart_tokenizer.save_pretrained("bart-tokenizer")

    return (pegasus_model, pegasus_tokenizer, score_pegasus), (bart_model, bart_tokenizer, score_bart)

In [18]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="text", column_summary="summary"):
    def generate_batch_sized_chunks(list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    score = metric.compute()
    return score

In [19]:
# Train and evaluate both models
(pegasus_model, pegasus_tokenizer, pegasus_score), (bart_model, bart_tokenizer, bart_score) = train_and_evaluate_models()

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
100,2.357000,2.339068
200,2.283800,2.158914
300,2.526900,2.104371
400,2.283700,2.073403
500,1.940500,2.057004
600,1.935500,2.048563
700,2.198100,2.043863


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
<ipython-input-17-70a4eaeb42b5>:23: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.

...

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.919800,2.186247
200,1.817300,2.176530
300,2.199000,2.021796
400,1.895800,2.026770
500,1.651000,1.987621
600,1.634000,1.952994
700,1.610900,1.916393


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
100%|██████████| 300/300 [07:39<00:00,  1.53s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size

        rouge1  rouge2    rougeL  rougeLsum
BART  0.017554  0.0003  0.017393   0.017321


In [20]:
# Prediction example using both models
def predict_with_models(text, pegasus_model, bart_model, pegasus_tokenizer, bart_tokenizer, gen_kwargs):
    pegasus_summary = pegasus_model.generate(pegasus_tokenizer(text, return_tensors="pt", truncation=True).input_ids.to(device), **gen_kwargs)
    bart_summary = bart_model.generate(bart_tokenizer(text, return_tensors="pt", truncation=True).input_ids.to(device), **gen_kwargs)

    pegasus_summary_text = pegasus_tokenizer.decode(pegasus_summary[0], skip_special_tokens=True)
    bart_summary_text = bart_tokenizer.decode(bart_summary[0], skip_special_tokens=True)

    return pegasus_summary_text, bart_summary_text

In [21]:
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}
sample_text = test_dataset[0]["text"]
reference = test_dataset[0]["summary"]

pegasus_summary, bart_summary = predict_with_models(sample_text, pegasus_model, bart_model, pegasus_tokenizer, bart_tokenizer, gen_kwargs)

print("Text:")
print(sample_text)
print("\nReference Summary:")
print(reference)
print("\nPEGASUS Summary:")
print(pegasus_summary)
print("\nBART Summary:")
print(bart_summary)

Text:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremo